### 트랜잭션 처리

#### (pycharm terminal) 앱추가
```
python manage.py startapp transaction
```

#### config/settings.py 추가, (**순서중요)
```config/settings.py
INSTALLED_APPS = [
    ...
    'transaction',
]
```

#### transaction/models.py
```transaction/models.py
from django.db import models

class Emp(models.Model):
    empno = models.IntegerField(primary_key=True)
    ename = models.CharField(max_length=50, null=False)
    deptno = models.IntegerField(null=False)
```

#### (pycharm terminal)
```
python manage.py makemigrations
python manage.py migrate
```

#### (pycharm terminal) 웹서버구동
```
python manage.py runserver
```

#### transaction/urls.py
```
from django.urls import path
from transaction import views

urlpatterns = [
    path('', views.home),
    path('insert', views.insert),
    path('list', views.list),
]
```

#### config/urls.py
```config/urls.py
urlpatterns = [
    ...
    path('transaction/', include('transaction.urls')),
]
```

#### transaction/views.py
```transaction/views.py
from django.shortcuts import render
import time
from django.db import transaction
from transaction.models import Emp

def home(request):
    return render(request, 'transaction/index.html')

@transaction.atomic()   # transaction을 처리하겠다 :: 1000개의 레코드 추가. 모두 성공해야 끝남.
# 하나라도 실패하면 전체 취소
def insert(request):
    start = time.time()          # for문 돌리기 전 시각
    Emp.objects.all().delete()   # 모든레코드 삭제
    for i in range(1,1001):      # 레코드 1~1000개 만들기
        emp = Emp(empno=i, ename='name' + str(i), deptno=i)
        emp.save()               #
    end = time.time()            # for문 돌린 후 시각
    runtime = end - start

    cnt = Emp.objects.count()    # 레코드수 카운트
    return render(request, 'transaction/index.html',
                  {'cnt':cnt, 'runtime': f'{runtime:.2f}초'})

def list(request):
    empList = Emp.objects.all().order_by('empno')
    return render(request, 'transaction/list.html',
                  {'empList': empList, 'empCount': len(empList)})
```

### (cmd)
```
sqlplus python/1234
(sql) desc transaction_emp;
(sql) insert into transaction_emp values (1, 'kim' 10);
(sql) select * from transaction_emp;
```
cmd 강제종료 후 재접속 시, insert가 반영이 안 되어 있음
why: 바로 종료할 경우 rollback; 변경사항 취소
commit; 명령어를 실행해야 변경사항을 확정함


### (cmd)
```
sqlplus python/1234
(sql) insert into transaction_emp values (1, 'kim' 10);
(sql) select * from transaction_emp;
(sql) commit;
```
cmd종료 후 재접속 시, 확인이 됨
파이썬에는 auto commit 이 되나, cmd에서는 commit 명령어 별도 입력해야 함

**참고 (sql) exit; 하면 auto commit이 됨


#### address/templates/index.html
```address/templates/index.html
...
<a href="transaction/">트랜잭션 처리</a><br>
```

#### transaction/templates/transaction/index.html
```transaction/templates/transaction/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
{% if cnt>0 %}
  {{cnt}}건의 자료가 추가되었습니다. <br>
  실행시간: {{runtime}} <br>
  <a href="list">확인</a><br>
{% endif %}
<a href="insert">트랜잭션 처리 실행</a><br>
<a href="/">Home</a>
</body>
</html>
```

#### transaction/templates/transaction/list.html
```transaction/templates/transaction/list.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
사원수:{{empCount}}명
<input type="button" value="트랜잭션 메인" onclick="location.href='/transaction'">
<input type="button" value="Home" onclick="location.href='/'">
<table border="1" width="600px">
  <tr>
    <td>사번</td>
    <td>이름</td>
    <td>부서</td>
  </tr>
  {% for emp in empList %}
  <tr>
    <td>{{emp.empno}}</td>
    <td>{{emp.ename}}</td>
    <td>{{emp.deptno}}</td>
  </tr>
  {% endfor %}
</table>
</body>
</html>
```
